In [1]:
import copy
import csbgnpy.pd.io.cd
import csbgnpy.pd.io.sbgnml
import csbgnpy.pd.io.sbgntxt
from csbgnpy.pd.compartment import Compartment

In [2]:
CYCLE_MODELFILE = "maps/RB_E2F.xml"
CLOCK_MODELFILE = "maps/CircadianClock.xml"

In [3]:
cycle = csbgnpy.pd.io.cd.read(CYCLE_MODELFILE)
clock = csbgnpy.pd.io.cd.read(CLOCK_MODELFILE)

# CYCLE

## Addition of transcriptional effects

When commented, only E2F1 associated to DP1 or DP1 and RB phosphorylated three times has transcriptional effects (we consider them as the most active forms of E2F1). This model corresponds to trigger2.

In [4]:
sources = []
targets = []
        
sources.append(("Complex(["
            "SubMacromolecule([][][@]SWI/SNF)|"
            "SubMacromolecule([][][]HDAC1)|"
            "SubMacromolecule([][][@]DP1*)|"
            "SubMacromolecule([][][@Ser364|@Ser31|@|@Ser403|@Lys]E2F1)|"
            "SubMacromolecule([][][@|@|@|P@]pRB*)]"
            "[][]pRB*/E2F1/DP1*/SWI/SNF/HDAC1#"
            "Compartment(nucleus))"))

sources.append(("Complex(["
            "SubMacromolecule([][][@]SWI/SNF)|"
            "SubMacromolecule([][][@]DP1*)|"
            "SubMacromolecule([][][@Ser364|@Ser31|@|@Ser403|@Lys]E2F1)|"
            "SubMacromolecule([][][@|@|P@|P@]pRB*)]"
            "[][]pRB*/E2F1/DP1*/SWI/SNF#"
            "Compartment(nucleus))"))

sources.append(("Complex(["
            "SubMacromolecule([][][@]DP1*)|"
            "SubMacromolecule([][][@Ser364|@Ser31|@|@Ser403|@Lys]E2F1)|"
            "SubMacromolecule([][][@|P@|P@|P@]pRB*)]"
            "[][]pRB*/E2F1/DP1*#"
            "Compartment(nucleus))"))

sources.append(("Complex(["
            "SubMacromolecule([][][@]DP1*)|"
            "SubMacromolecule([][][@Ser364|@Ser31|@|@Ser403|@Lys]E2F1)]"
            "[][]E2F1/DP1*#"
            "Compartment(nucleus))"))

for source in sources:
    assert cycle.get_entity(source, by_string = True)

targets.append("Macromolecule([][@Thr380]cyclin E1*#Compartment(nucleus))")

targets.append("Macromolecule([][]cyclin A2*#Compartment(nucleus))")

targets.append("Macromolecule([][]cyclin B1*)")

for target in targets:
    assert cycle.get_entity(target, by_string = True)


emptyset = "EmptySet()"

for i, target in enumerate(targets):
    p = "GenericProcess([{}][{}])".format(emptyset, target)
    cycle.add_process(p)
    for i, source in enumerate(sources):
        if not "cyclin A2*" in target or i == 2 or i == 3:
            m = "Stimulation({}|{})".format(source, p)
            cycle.add_modulation(m)

In [5]:
!mkdir -p generated
csbgnpy.pd.io.sbgnml.write(cycle, "generated/RBE2F_bmc_triggers.sbgnml")

## Re-labelling

### Re-labelling of compartements

The "nucleus" compartement is renamed to "nucleoplasm". In CYCLE, we add the compartment "cytoplasm" to all EPNs that are in no compartment, to match with CLOCK

In [6]:
cycle.query_compartments("nucleus")[0].label = "nucleoplasm"
cytoplasm = Compartment(label="cytoplasm")
cycle.add_compartment(cytoplasm)
for e in cycle.entities:
    if e.compartment is None:
        e.compartment = cytoplasm

### Re-labelling of entities

We rename entities to match with CLOCK

In [7]:
cycle.replace_label("p16INK4a\*", "p16INK4a")

In [8]:
cycle.replace_label("p53\*", "p53")

In [9]:
cycle.replace_label("p21Cip*", "p21Cip")

In [10]:
cycle.replace_label("cyclin B1\*", "CCNB1")

In [11]:
cycle.replace_label("CDC2/", "CDK1/")

In [12]:
cycle.replace_label("CDC2$", "CDK1")

In [13]:
cycle.replace_sv_var("Ser 1981", "Ser1981")

In [14]:
cycle.replace_label("CDK1/CCNB1", "CCNB1/CDK1")



## Remove redundant entities

### Remove p53/MDM2 complexes

The process is better described in CLOCK.
Complexes in both compartments should be removed.

In [15]:
for e in cycle.query_entities("p53/MDM2"):
    cycle.remove_entity(e)

##  Sites

### Sites of p53

We extend the sites of p53 with two undefined state variables (unset value, undefined variable) for ubiquitination

In [16]:
for e in cycle.query_entities("\[.*\|.*\]p53"):
    e.svs += [csbgnpy.pd.sv.StateVariable(var = csbgnpy.pd.sv.UndefinedVar(1)), 
                    csbgnpy.pd.sv.StateVariable(var = csbgnpy.pd.sv.UndefinedVar(2))]

## Replace WEE1

In [17]:
cycle.replace_entity("Macromolecule([][@]WEE1#Compartment(nucleoplasm))", "Macromolecule([][@Ser53|@Ser472]WEE1#Compartment(nucleoplasm))")
cycle.add_modulation("Catalysis(Macromolecule([][P@S317|P@S345]CHEK1#Compartment(nucleoplasm))|GenericProcess([1:Macromolecule([][@Ser53|@Ser472]WEE1#Compartment(nucleoplasm))][1:Macromolecule([][@Ser53|P@Ser472]WEE1#Compartment(nucleoplasm))]))")
cycle.add_modulation("Catalysis(Complex([SubMacromolecule([][]CCNB1)|SubMacromolecule([][@Tyr15|P@Thr161|@Thr14]CDK1)][][]CCNB1/CDK1#Compartment(nucleoplasm))|GenericProcess([1:Macromolecule([][@Ser53|@Ser472]WEE1#Compartment(nucleoplasm))][1:Macromolecule([][P@Ser53|@Ser472]WEE1#Compartment(nucleoplasm))]))")
cycle.add_modulation("Catalysis(Macromolecule([][@Ser53|P@Ser472]WEE1#Compartment(nucleoplasm))|GenericProcess([1:Complex([SubMacromolecule([][@Tyr15|@Thr161|@Thr14]CDK1)|SubMacromolecule([][]CCNB1)][][]CCNB1/CDK1#Compartment(nucleoplasm))][1:Complex([SubMacromolecule([][]CCNB1)|SubMacromolecule([][P@Tyr15|@Thr161|@Thr14]CDK1)][][]CCNB1/CDK1#Compartment(nucleoplasm))]))")
cycle.remove_entity("Macromolecule([][P@]WEE1#Compartment(nucleoplasm))")

## Fix DP1

In [18]:
p = cycle.query_processes("\[1:Macromolecule.*DP1.*cytoplasm.*\]\[1:Macromolecule.*DP1.*nucleoplasm.*\]")[0]
cycle.remove_process(p)

p = "GenericProcess([1:Macromolecule([][@]DP1*#Compartment([]nucleoplasm))][1:Macromolecule([][@]DP1*#Compartment([]cytoplasm))])"
cycle.add_process(p)

p = cycle.query_processes("\[1:Macromolecule.*P.*DP1.*\]\[1:Macromolecule.*DP1.*\]")[0]
mod = "Catalysis(Macromolecule(PP2A#Compartment(nucloeplasm))|{})".format(p)
cycle.add_modulation(mod)

# CLOCK

## Add effect of PP-p53 on transcription of p21

Transcription of p21 is only in CLOCK, PP-p53 is only in CYCLE.

In [19]:
mods = clock.query_modulations("\[.*\]\[1:NucleicAcidFeature\(.*p21cip.*\)\]")
for mod in mods:
    clock.remove_modulation(mod)

p = "GenericProcess([1:NucleicAcidFeature([][]p21cip#Compartment([]nucleoplasm))][1:NucleicAcidFeature([RNA][]p21cip#Compartment([]cytoplasm))])"
p53 = "Macromolecule([][@|@|P@Ser15|P@Ser20]p53#Compartment([]nucleoplasm))"
rorg = "Macromolecule([][]RORG#Compartment([]nucleoplasm))"
nr1d1 = "Macromolecule([][@T274|P@S55|@]NR1D1#Compartment([]nucleoplasm))"
p21gene = "NucleicAcidFeature([][]p21cip#Compartment([]nucleoplasm))"
mod = "NecessaryStimulation(AndOperator([{}|OrOperator([{}|AndOperator([{}|NotOperator([{}])])])])|{})".format(p21gene, p53, rorg, nr1d1, p)

assert clock.get_process(p, by_string=True) is not None
assert cycle.get_entity(p53, by_string=True) is not None # PP-p53 is only in CYCLE
assert clock.get_entity(rorg, by_string=True) is not None
assert clock.get_entity(nr1d1, by_string=True) is not None
assert clock.get_entity(p21gene, by_string=True) is not None

clock.add_modulation(mod)
print(str(clock.get_modulation(mod, by_string=True)))

NecessaryStimulation(AndOperator([NucleicAcidFeature([][]p21cip#Compartment([]nucleoplasm))|OrOperator([AndOperator([Macromolecule([][]RORG#Compartment([]nucleoplasm))|NotOperator([Macromolecule([][@|P@S55|@T274]NR1D1#Compartment([]nucleoplasm))])])|Macromolecule([][@|@|P@Ser15|P@Ser20]p53#Compartment([]nucleoplasm))])])|GenericProcess([1:NucleicAcidFeature([][]p21cip#Compartment([]nucleoplasm))][1:NucleicAcidFeature([RNA][]p21cip#Compartment([]cytoplasm))]))


## Modify logical function that stimulates transcription of Per2

Production of mRNA-PER2 is stimulated by the logical function: GC or HSF1 or (CLOCK-BMAL1 and not PC)

In [20]:
for e in clock.entities:
    if "BMAL1" in str(e) and "PER2" in str(e):
        print(e)

Complex([SubMacromolecule([][@|@S121|@S661]PER1)|SubMacromolecule([][@|@S280|@S71]CRY1)|SubMacromolecule([][@|@S558]CRY2)|SubMacromolecule([][@|Ac@L538|@S17|P@S90|@T21]BMAL1)|SubMacromolecule([][ADPr@]CLOCK)|SubMacromolecule([][Ac@|@|@S662]PER2)|SubMacromolecule([][]CSNK1δ)|SubMacromolecule([][]CSNK1ε)][][]PER/CRY/CSNK1/CLOCK/BMAL1#Compartment([]nucleoplasm))


In [21]:
mods = clock.query_modulations("\(.*\|GenericProcess\(\[.*\]\[1:.*\[RNA\]\[\]Per2#Compartment\(\[\]cytoplasm\)\)\]\)\)")

proc = mods[0].target

for m in mods:
    clock.remove_modulation(m)
      
gc = "Complex([SubMacromolecule([receptor][Ac@lys480]GCR)|SubSimpleChemical([][]GC)][][]GCR/GC#Compartment([]nucleoplasm))"
hsf1 = "Macromolecule([][]HSF1#Compartment(nucleoplasm))"
bmal_clock = "Complex([SubMacromolecule([][@T21|@S17|P@S90|Ac@L538|@]BMAL1)|SubMacromolecule([][ADPr@]CLOCK)|SubMacromolecule([][]CK2α)][][]BMAL1/CLOCK/CK2α#Compartment([]nucleoplasm))"
pc = "Complex([SubMacromolecule([][@S280|@S71|@]CRY1)|SubMacromolecule([][@S558|@]CRY2)|SubMacromolecule([][@S661|@S121|@]PER1)| \
        SubMacromolecule([][@T21|@S17|P@S90|Ac@L538|@]BMAL1)|SubMacromolecule([][ADPr@]CLOCK)| \
        SubMacromolecule([][Ac@|@S662|@]PER2)|SubMacromolecule([][]CSNK1δ)| \
        SubMacromolecule([][]CSNK1ε)][][]PER/CRY/CSNK1/CLOCK/BMAL1#Compartment([]nucleoplasm))"

assert clock.get_entity(gc, by_string=True) is not None
assert clock.get_entity(hsf1, by_string=True) is not None
assert clock.get_entity(bmal_clock, by_string=True) is not None
assert clock.get_entity(pc, by_string=True) is not None

lo = "OrOperator([{}|{}|AndOperator([{}|NotOperator([{}])])])".format(gc, hsf1, bmal_clock, pc)
mod = "Stimulation({}|{})".format(lo, proc)

clock.add_modulation(mod)
print(str(clock.get_modulation(mod, by_string=True)))

Stimulation(OrOperator([AndOperator([Complex([SubMacromolecule([][@|Ac@L538|@S17|P@S90|@T21]BMAL1)|SubMacromolecule([][ADPr@]CLOCK)|SubMacromolecule([][]CK2α)][][]BMAL1/CLOCK/CK2α#Compartment([]nucleoplasm))|NotOperator([Complex([SubMacromolecule([][@|@S121|@S661]PER1)|SubMacromolecule([][@|@S280|@S71]CRY1)|SubMacromolecule([][@|@S558]CRY2)|SubMacromolecule([][@|Ac@L538|@S17|P@S90|@T21]BMAL1)|SubMacromolecule([][ADPr@]CLOCK)|SubMacromolecule([][Ac@|@|@S662]PER2)|SubMacromolecule([][]CSNK1δ)|SubMacromolecule([][]CSNK1ε)][][]PER/CRY/CSNK1/CLOCK/BMAL1#Compartment([]nucleoplasm))])])|Complex([SubMacromolecule([receptor][Ac@lys480]GCR)|SubSimpleChemical([][]GC)][][]GCR/GC#Compartment([]nucleoplasm))|Macromolecule([][]HSF1#Compartment([]nucleoplasm))])|GenericProcess([1:NucleicAcidFeature([][]Per2#Compartment([]nucleoplasm))][1:NucleicAcidFeature([RNA][]Per2#Compartment([]cytoplasm))]))


## Replace inhibitions by absolute inhibitions

Default logic rule is: at least one activator and no inhibitor.

In [22]:
mods = clock.query_modulations("Inhibition")
for m in mods:
    clock.replace_modulation(m, str(m).replace("Inhibition", "AbsoluteInhibition"))    

# MERGE

In [23]:
merge = cycle.union(clock)

## Renew IDs

In [24]:
merge.renew_ids()
inter = clock.intersection(cycle)
clock_new = merge.difference(cycle.difference(inter))
cycle_new = merge.difference(clock.difference(inter))

assert cycle == cycle_new
assert clock == clock_new

## Write SBGN-ML files

In [25]:
csbgnpy.pd.io.sbgnml.write(cycle_new, "generated/CYCLE.sbgnml")
csbgnpy.pd.io.sbgnml.write(clock_new, "generated/CLOCK.sbgnml")
csbgnpy.pd.io.sbgnml.write(merge, "generated/MERGE.sbgnml")

## Write CD files

In [26]:
nets = [(cycle, cycle_new), (clock, clock_new)]
files = [(CYCLE_MODELFILE, "generated/CYCLE.xml"), (CLOCK_MODELFILE, "generated/CLOCK.xml")]
rep = []
for i, couple in enumerate(nets):
    rep.append({})
    for c in couple[0].compartments:
        cc = couple[1].get_compartment(c, by_compartment = True)
        rep[i][str(c.id)] = str(cc.id) if cc else None
    for e in couple[0].entities:
        ee = couple[1].get_entity(e, by_entity = True)
        rep[i][str(e.id)] = str(ee.id)  if ee else None
        if hasattr(e, "svs"):
            for sv in e.svs:
                svsv = ee.get_sv(sv, by_sv = True) if ee else None
                rep[i][str(sv.id)] = str(svsv.id) if svsv else None
for i, couple in enumerate(files):
    f = open(files[i][0])
    s = f.read()
    f.close()
    for k in rep[i].keys():
        s = s.replace('"{}"'.format(k), '"{}"'.format(rep[i][k]))
        s = s.replace('"{},'.format(k), '"{},'.format(rep[i][k]))
        s = s.replace(',{}"'.format(k), ',{}"'.format(rep[i][k]))
        s = s.replace(',{},'.format(k), ',{},'.format(rep[i][k]))
        s = s.replace(">{}<".format(k), ">{}<".format(rep[i][k]))
    f = open(files[i][1], 'w')
    f.write(s)
    f.close()